# DEDS WC 4.2

## Imports

In [5]:
import pandas as pd
import sqlite3 as sql3
import pyodbc

## Connecties sqlite databases

In [9]:
crm_conn = sql3.connect(r"../../../data/raw/go_crm_train.sqlite")
sales_conn = sql3.connect(r"../../../go_sales_volledig.sqlite")
staff_conn = sql3.connect(r"../../../data/raw/go_staff_volledig.sqlite")

## Connectie SDM

In [6]:
DB = {
    "servername" : r"VISSIE\SQLEXPRESS",
    "database" : "SDM"}

export_conn = pyodbc.connect(f"""DRIVER={'SQL SERVER'};
                             SERVER={DB['servername']};
                             DATABASE={DB['database']};
                             Trusted_Connection=yes
                             """)

export_conn.setencoding('utf-8')
export_conn.setdecoding(pyodbc.SQL_CHAR, encoding='utf-8')
export_conn.setdecoding(pyodbc.SQL_WCHAR, encoding='utf-8')

export_cursor = export_conn.cursor()

## Automatische dataframes

In [10]:
def create_dataframes_sql(connection, db_type : str):
    dictionary : dict = {}
    query : str = ""
    key : str = ""
    
    if (db_type == "sqlite"):
        query = "SELECT name FROM sqlite_master WHERE type='table';"
        key = "name"
    elif (db_type == "ssms"):
        query = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE';"
        key = "TABLE_NAME"
    else:
        return {}
    table_names = pd.read_sql(query, connection)
    
    for table in table_names[key].tolist():
        dictionary[table] = pd.read_sql(f"SELECT * FROM {table}", connection)
    
    return dictionary

def create_dataframes_csv(path, name):
    frame = pd.read_csv(path)
    
    return {name : frame}

## Uitlezen connecties pandas

In [16]:
forecast = create_dataframes_csv(r"../../../data/raw/updated_product_forecast_volledig.csv", "forecast")
inventory = create_dataframes_csv(r"../../../data/raw/inventory_levels_volledig.csv", "inventory")
crm = create_dataframes_sql(crm_conn, "sqlite")
sales = create_dataframes_sql(sales_conn, "sqlite")
staff = create_dataframes_sql(staff_conn, "sqlite")

forecast["forecast"]



,FORECAST_ID,PRODUCT_NUMBER,YEAR,MONTH,EXPECTED_VOLUME
0,1,44,2024,12,383
1,2,45,2023,1,80
2,3,45,2023,2,51
3,4,45,2023,3,214
4,5,45,2023,4,300
...,...,...,...,...,...
3867,3868,115,2024,8,282
3868,3869,115,2024,9,920
3869,3870,115,2024,10,1081
3870,3871,115,2024,11,398


## 

## Forecast ids toevoegen

In [12]:
# file_path = r"..\..\..\data\raw\product_forecast_train.csv" 
# df = pd.read_csv(file_path)

# df.insert(0, 'FORECAST_ID', range(1, len(df) + 1))

# df.to_csv(r"..\..\..\data\raw\updated_product_forecast_train.csv", index=False)

# print("DONE: FORECAST_ID added at the beginning.")


DONE: FORECAST_ID added at the beginning.
